# Group Comparison analysis pRF propertis 

In [1]:
# Load the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 
import shutil
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Define whether you want to run the code on one subject or on the whole dataset

# 1. Run the code on one specific subject 
#subjects = ['sub-22']

# 2. Run the code on glaucoma patients 
#subjects = [f'sub-{i:02}'for i in range(2, 21)]

# 3. Run the code on healthy controls 
# #subjects = [f'sub-{i:02}'for i in range(21, 47)]

In [4]:
# Define the main variables 
target = 'V3'
source = 'V1'
hemis = ['lh', 'rh']
atlas = 'manual'
delineation = 'manualdelin'
denoising = 'nordic'
task = 'RET'
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA/derivatives/CFM_demean'
poag = [f'sub-{i:02}' for i in range(2, 21)]
hc = [f'sub-{i:02}' for i in range(21, 47)]
subjects = poag + hc
visual_areas = ['V2', 'V3']
freesurfer = f"{MAIN_PATH}/freesurfer"
cumulative = os.path.join(MAIN_PATH,'culumativeplots')
sigmas = []
if os.path.exists(cumulative): 
    shutil.rmtree(cumulative)
os.makedirs(cumulative)
pdf_hemis = PdfPages(os.path.join(cumulative, 'hemis.pdf'))
pdf_hemi = PdfPages(os.path.join(cumulative, 'hemi.pdf'))

In [5]:
# PLOT THE CUMULATIVE DISTRIBUTION FUNCTION REPRESENTING THE CONNECTIVE FIELD SIZE SUBJECT PER SUBJECTS

for subj in subjects: 
    sigma = {}

    for hemi in hemis: 
        best_fit_path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{target}-{source}/best_fits_prf.csv'
        if not os.path.exists(best_fit_path):
            print(f'Best fit file not found for {subj} ({hemi}).')
            continue

        best_fit_df = pd.read_csv(best_fit_path)
        best_fit_df = best_fit_df[best_fit_df["Best Sigma Finer"] > 0.05]
        best_fit_df = best_fit_df[best_fit_df["Best Variance Explained Finer"] > 0.3]
        sigma[hemi] = best_fit_df["Best Sigma Finer"].values

    # Plot the hemispheres separately 
    fig_one, axs = plt.subplots(1, 2, figsize=(14, 6))
    for i in range(len(hemis)):
        hemi = hemis[i]
        if hemi in sigma:
            axs[i].hist(sigma[hemi], bins=50, cumulative=True, density=True, alpha=0.7)
            axs[i].set_title(f"{subj} - {hemi} - {target}-{source}")
            plt.xlim(0, max(max(sigma[hemi]), 1))
            axs[i].set_xlabel("Connective Field Sigma (degrees)")
            axs[i].set_ylabel("Cumulative Probability")
            axs[i].grid(True)

    fig_one.tight_layout()
    pdf_hemis.savefig(fig_one)
    plt.close(fig_one)

    # Plot the hemispheres together  
    if all(h in sigma for h in hemis):
        sigma_total = list(sigma['lh']) + list(sigma['rh']) 
        fig_two = plt.figure(figsize=(7, 6))
        plt.hist(sigma_total, bins=50, cumulative=True, density=True, alpha=0.7, color='blue')
        plt.title(f"{subj} - {target}-{source}")
        plt.xlim(0, max(sigma_total), 1)
        plt.xlabel("Connective Field Sigma (degrees)")
        plt.ylabel("Cumulative Probability")
        plt.grid(True)
        pdf_hemi.savefig(fig_two) 
        plt.close(fig_two)

pdf_hemis.close()
pdf_hemi.close()

/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)


Best fit file not found for sub-03 (lh).
Best fit file not found for sub-03 (rh).


/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: Matplotl

Best fit file not found for sub-21 (rh).


/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)


...

/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)


Best fit file not found for sub-28 (lh).
Best fit file not found for sub-28 (rh).


/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: Matplotl

Best fit file not found for sub-36 (lh).
Best fit file not found for sub-37 (lh).


/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: MatplotlibDeprecationWarning: Passing the emit parameter of set_xlim() positionally is deprecated since Matplotlib 3.6; the parameter will become keyword-only two minor releases later.
  plt.xlim(0, max(sigma_total), 1)
/var/folders/0k/zjsxn2197mdbzvx14lj8m0v00000gn/T/ipykernel_31673/3542458851.py:39: Matplotl

In [6]:
pdf_groupcomparison = PdfPages(os.path.join(cumulative, 'groupcomparison.pdf'))

def sigma(subj, visual_area):
    sigma = []
    for hemi in hemis:
        path = f'{MAIN_PATH}/{subj}/ses-1/GM/{hemi}/{visual_area}-{source}/best_fits_prf.csv'
        if not os.path.exists(path): continue
        df = pd.read_csv(path)
                # Convert to numeric, coercing bad values to NaN
        df["Best Sigma Finer"] = pd.to_numeric(df["Best Sigma Finer"], errors='coerce')
        df["Best Variance Explained Finer"] = pd.to_numeric(df["Best Variance Explained Finer"], errors='coerce')

        # Drop rows with any NaN in these columns (i.e., non-numeric or missing)
        df = df.dropna(subset=["Best Sigma Finer", "Best Variance Explained Finer"])

        df = df[(df["Best Sigma Finer"] > 0.05) & (df["Best Variance Explained Finer"] > 0.3)]
        sigma.extend(df["Best Sigma Finer"].values)
    return sigma

def individual_cdfs(group, color, label, visual_area):
    cdfs = []
    for subj in group:
        sigma_vals = sigma(subj, visual_area)
        if not sigma_vals:
            continue
        sorted_sigma = np.sort(sigma_vals)
        cdf = np.linspace(0, 1, len(sorted_sigma))
        plt.plot(sorted_sigma, cdf, linestyle='--', color=color, alpha=0.4)
        grid_cdf = np.interp(x, sorted_sigma, cdf, left=0, right=1)
        cdfs.append(grid_cdf)
    if cdfs:
        median = np.median(cdfs, axis=0)
        plt.plot(x, median, color=color, linewidth=2.5, label=label + " Median")
    return cdfs

for visual_area in visual_areas:
    x = np.linspace(0, 10, 500)  # Common sigma range for interpolation

    # Plot the healthy controls
    plt.figure(figsize=(5, 6))
    hc_cdfs = individual_cdfs(hc, 'blue', 'HC', visual_area)
    plt.title(f"{visual_area} Healthy Controls")
    plt.xlabel("Connective Field Sigma (degrees)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    plt.legend()
    pdf_groupcomparison.savefig()
    plt.close()

    # Plot the patients 
    plt.figure(figsize=(5, 6))
    poag_cdfs = individual_cdfs(poag, 'red', 'POAG', visual_area)
    plt.title(f"{visual_area} Glaucoma Patients")
    plt.xlabel("Connective Field Sigma (degrees)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    plt.legend()
    pdf_groupcomparison.savefig()
    plt.close()

    # Group comparison 
    plt.figure(figsize=(5, 6))
    if hc_cdfs:
        plt.plot(x, np.median(hc_cdfs, axis=0), color='blue', linewidth=2.5, label='HC Median')
    if poag_cdfs:
        plt.plot(x, np.median(poag_cdfs, axis=0), color='red', linewidth=2.5, label='POAG Median')
    plt.title(f"{visual_area} Group Median Comparison")
    plt.xlabel("Connective Field Sigma (degrees)")
    plt.ylabel("Cumulative Probability")
    plt.grid(True)
    plt.legend()
    pdf_groupcomparison.savefig()
    plt.close()

pdf_groupcomparison.close()